<a href="https://colab.research.google.com/github/Izuho/Quantum/blob/main/path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 4.5 MB/s 
     |████████████████████████████████| 18.0 MB 373 kB/s 
     |████████████████████████████████| 240 kB 60.1 MB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 112 kB 60.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.6 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 943 kB 48.0 MB/s 
     |████████████████████████████████| 4.0 MB 31.0 MB/s 
     |████████████████████████████████| 112 kB 51.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.36.2-py3-none-any.whl size=11933 sha256=d04c5e01c8

In [ ]:
from qiskit import QuantumCircuit, execute, Aer, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator
from math import sqrt, pi

In [ ]:
def make_path_EG(point, qnum):

    # make unitary matrix to compute expected value G
    unitary = None
    if point == 0:
        unitary = QuantumCircuit(2)
        unitary.x(0)
        unitary.z(1)
        unitary = unitary.to_gate().control()
    elif point == qnum-1:
        unitary = QuantumCircuit(2)
        unitary.z(0)
        unitary.x(1)
        unitary = unitary.to_gate().control()
    elif 1 <= point and point <= qnum-2:
        unitary = QuantumCircuit(3)
        unitary.z(0)
        unitary.x(1)
        unitary.z(2)
        unitary = unitary.to_gate().control()
    else:
        raise Exception('OUT of BOUNDS!')

    # make path grath state
    qc = QuantumCircuit()
    qr = QuantumRegister(qnum+1,'qreg')
    qc.add_register(qr)
    for i in range(qnum+1):
        qc.h(qr[i])
    for i in range(0, qnum, 2):
        qc.cz(qr[i], qr[i+1])
    for i in range(1,qnum-1, 2):
        qc.cz(qr[i], qr[i+1])
    
    # compute expected value G
    if point == 0:
        qc.append(unitary, [qr[qnum], qr[0], qr[1]])
        qc.h(qr[qnum])
    elif point == qnum-1:
        qc.append(unitary, [qr[qnum], qr[point-1], qr[point]])
        qc.h(qr[qnum])
    else:
        qc.append(unitary, [qr[qnum], qr[point-1], qr[point], qr[point+1]])
        qc.h(qr[qnum])

    cr = ClassicalRegister(1,'creg')
    qc.add_register(cr)
    qc.measure(qr[qnum],cr[0])
    qc.draw()

    return qc

In [ ]:
def calculate_path_Bell(qnum):
    ans = 0
    for i in range(qnum):
        circ = make_path_EG(i, qnum)
        emulator = Aer.get_backend('qasm_simulator')
        job = execute(circ, emulator, shots=8192)
        hist = job.result().get_counts()

        ratio = None
        if hist.get('1') == None:
            ratio = 1
        else:
            ratio = (hist['0']-hist['1'])/(hist['0']+hist['1'])

        if i == 1 and qnum >= 3:
            ans += sqrt(2) * 2 * ratio
        elif 0 <= i and i <= 2:
            ans += sqrt(2) * ratio
        else:
            ans += ratio

    return ans

In [ ]:
N = 20
ratio = calculate_path_Bell(N)
print('Classical Bound  : ',max(N-1,2),'\nQuantum Bound : ',ratio)

Classical Bound  :  19 
Quantum Bound :  22.65685424949238
